In [1]:
import pandas as pd
from Acesso import Login
from Query import Query
from glob import glob
from datetime import datetime,timedelta

s=Login()

sql=Query(s.usuario,s.senha,s.database,s.server)

pd.set_option('display.max_columns',None)

In [2]:
querys={
    
    'Produto':
    
    """
    
    SELECT * FROM netfeira.vw_produto
    
    """,
    
    'Cliente':
    
    """
    
    SELECT * FROM netfeira.vw_cliente
    
    """,
    
    'Segmento':
    
    """
    
    SELECT * FROM netfeira.vw_segmento
    
    """,
    
    'Vendas':
    
    """
    
    SELECT * FROM netfeira.vw_targetestatistico
    
    """
}

In [4]:
dt_atu=datetime.now()

dt_ant=dt_atu-timedelta(days=90)

In [5]:
tabelas_df=sql.CriarTabela(kwargs=querys)

In [6]:
tabelas_df['Consolidado']=tabelas_df['Vendas'].loc[(tabelas_df['Vendas']['ID Situação']=='FA')&(tabelas_df['Vendas']['Tipo de Operação']=='VENDAS')&(tabelas_df['Vendas']['Data de Faturamento'].between(dt_ant,dt_atu))]

In [7]:
tabelas_df['Cliente']=tabelas_df['Cliente'].merge(tabelas_df['Segmento'],on='ID Segmento',how='inner')

In [8]:
tabelas_df['Consolidado']=tabelas_df['Consolidado'].merge(tabelas_df['Cliente'][['ID Cliente','Segmento','Canal']].loc[tabelas_df['Cliente']['Status do Cliente']=='ATIVO'],on='ID Cliente',how='inner')

In [9]:
def Classificacao(val):
    
    if(val<=80):
        
        tipo='A'
        
        pass
    
    elif(val<=95):
        
        tipo='B'
        
        pass
    
    
    else:
        
        tipo='C'
        
        pass
    
    return tipo    
    
    pass

In [10]:
tabelas_df['Analise']=tabelas_df['Consolidado'][['SKU','Pedido']].groupby(['SKU'],as_index=False).count()

tabelas_df['Analise'].sort_values('Pedido',ascending=False,ignore_index=True,inplace=True)

valores=[]

soma=0

for val in tabelas_df['Analise']['Pedido'].tolist():
    
    soma+=val
    
    valores.append(soma)
    
    pass

tabelas_df['Analise']['Acumulado']=valores

total=tabelas_df['Analise']['Pedido'].sum()

tabelas_df['Analise']['Total Geral']=total

tabelas_df['Analise']['Perc']=round(tabelas_df['Analise']['Acumulado']/tabelas_df['Analise']['Total Geral'],4)*100

tabelas_df['Analise']['Classificação']=tabelas_df['Analise']['Perc'].apply(Classificacao)

tabelas_df['Analise'].loc[tabelas_df['Analise']['Classificação']=='A']

,SKU,Pedido,Acumulado,Total Geral,Perc,Classificação
0,139,2229,2229,54140,4.12,A
1,158,1573,3802,54140,7.02,A
2,440,1185,4987,54140,9.21,A
3,3,1148,6135,54140,11.33,A
4,401,1113,7248,54140,13.39,A
...,...,...,...,...,...,...
93,9567,163,42594,54140,78.67,A
94,216,163,42757,54140,78.97,A
95,9448,162,42919,54140,79.27,A
96,1058,160,43079,54140,79.57,A


,SKU,Pedido,Acumulado,Total Geral,Perc,Classificação
0,139,2229,2229,54140,4.12,A
1,158,1573,3802,54140,7.02,A
2,440,1185,4987,54140,9.21,A
3,3,1148,6135,54140,11.33,A
4,401,1113,7248,54140,13.39,A
...,...,...,...,...,...,...
93,9567,163,42594,54140,78.67,A
94,216,163,42757,54140,78.97,A
95,9448,162,42919,54140,79.27,A
96,1058,160,43079,54140,79.57,A


In [11]:
segmentos=tabelas_df['Segmento']['Segmento'].unique().tolist()

temp_df=pd.DataFrame()

for s in segmentos:
    
    df=pd.DataFrame()
    
    df=tabelas_df['Consolidado'][['Segmento','Canal','SKU','Pedido']].loc[tabelas_df['Consolidado']['Segmento']==s].groupby(['Segmento','Canal','SKU'],as_index=False).sum()
    
    df.sort_values('Pedido',ascending=False,ignore_index=True,inplace=True)
    
    valores=[]

    soma=0

    for val in df['Pedido'].tolist():

        soma+=val

        valores.append(soma)

        pass

    df['Acumulado']=valores

    total=df['Pedido'].sum()

    df['Total Geral']=total

    df['Perc']=round(df['Acumulado']/df['Total Geral'],4)*100

    df['Classificação']=df['Perc'].apply(Classificacao)
    
    temp_df=pd.concat([temp_df,df],axis=0,ignore_index=True)
    
    #break
    
    pass

temp_df=temp_df.merge(tabelas_df['Analise'][['SKU','Classificação']],on='SKU',how='inner')
    
temp_df['Igual']=temp_df.apply(lambda info: 1 if info['Classificação_x']==info['Classificação_y'] else 0,axis=1)
        
temp_df=temp_df.loc[(temp_df['Igual']==1)&(temp_df['Classificação_x']=='A')]

temp_df


,Segmento,Canal,SKU,Pedido,Acumulado,Total Geral,Perc,Classificação_x,Classificação_y,Igual
0,10 OU + CHK,AS,139,122877421,122877421.0,2955924315,4.16,A,A,1
1,EMPÓRIO,AS,139,5156906,29164591.0,150467476,19.38,A,A,1
2,MERCEARIA,AS,139,11995878,89259060.0,329448095,27.09,A,A,1
3,RESTAURANTE,FS,139,176513544,536450684.0,4324707680,12.40,A,A,1
4,CHURRASCARIA,FS,139,5150136,18006410.0,56561934,31.83,A,A,1
...,...,...,...,...,...,...,...,...,...,...
2902,SAÚDE,FS,414,8593474,57528337.0,252834185,22.75,A,A,1
2903,ENTRETENIMENTO,FS,414,2594185,233537197.0,321398573,72.66,A,A,1
2905,COZINHA INDUSTRIAL,FS,414,15911418,15911418.0,286305343,5.56,A,A,1
2906,EDUCAÇÃO,FS,414,21943297,126422280.0,587248086,21.53,A,A,1


,Segmento,Canal,SKU,Pedido,Acumulado,Total Geral,Perc,Classificação_x,Classificação_y,Igual
0,10 OU + CHK,AS,139,122877421,122877421.0,2955924315,4.16,A,A,1
1,EMPÓRIO,AS,139,5156906,29164591.0,150467476,19.38,A,A,1
2,MERCEARIA,AS,139,11995878,89259060.0,329448095,27.09,A,A,1
3,RESTAURANTE,FS,139,176513544,536450684.0,4324707680,12.40,A,A,1
4,CHURRASCARIA,FS,139,5150136,18006410.0,56561934,31.83,A,A,1
...,...,...,...,...,...,...,...,...,...,...
2902,SAÚDE,FS,414,8593474,57528337.0,252834185,22.75,A,A,1
2903,ENTRETENIMENTO,FS,414,2594185,233537197.0,321398573,72.66,A,A,1
2905,COZINHA INDUSTRIAL,FS,414,15911418,15911418.0,286305343,5.56,A,A,1
2906,EDUCAÇÃO,FS,414,21943297,126422280.0,587248086,21.53,A,A,1


In [12]:
prod_df=pd.DataFrame()

for s in segmentos:
    
    df=pd.DataFrame()
    
    df=temp_df.loc[temp_df['Segmento']==s].sort_values('Pedido',ascending=False,ignore_index=True)
        
    df=df.iloc[:5]
    
    prod_df=pd.concat([prod_df,df],axis=0,ignore_index=True)
    
    #break
    
    pass

contagem=len(prod_df['Segmento'].unique().tolist())

base_df=prod_df[['SKU','Segmento']].groupby(['SKU'],as_index=False).count()

base_df=base_df.merge(tabelas_df['Produto'][['SKU','Produto']],on='SKU',how='inner')[['SKU','Produto','Segmento']]

base_df.sort_values('Segmento',ascending=False,ignore_index=True,inplace=True)

base_df.loc[base_df['Segmento']>1]

,SKU,Produto,Segmento
0,139,POLPINHA DE MARACUJA (100G),18
1,440,"IQF MORANGO (1,2 KG)",17
2,34,"IQF AMORA CONGELADOS (1,200 KG)",9
3,95,"IQF FRAMBOESA CONGELADOS 1,200 KG",8
4,158,POLPINHA DE MORANGO (100G),8
5,1079,"IQF BLUEBERRIES CONGELADOS 1,200 KG",7
6,9596,CONG. BATATA MCCAIN 9MM TRAD 720G (16X720GR),6
7,9549,CONG. BATATA MCCAIN SMILES 400G (22X400GR),6
8,415,POLPA DE MARACUJA S/SEM (1020G),5
9,594,"CONGELADO BATATA MCCAIN 9MM TRADICIONAL 2,5",5


,SKU,Produto,Segmento
0,139,POLPINHA DE MARACUJA (100G),18
1,440,"IQF MORANGO (1,2 KG)",17
2,34,"IQF AMORA CONGELADOS (1,200 KG)",9
3,95,"IQF FRAMBOESA CONGELADOS 1,200 KG",8
4,158,POLPINHA DE MORANGO (100G),8
5,1079,"IQF BLUEBERRIES CONGELADOS 1,200 KG",7
6,9596,CONG. BATATA MCCAIN 9MM TRAD 720G (16X720GR),6
7,9549,CONG. BATATA MCCAIN SMILES 400G (22X400GR),6
8,415,POLPA DE MARACUJA S/SEM (1020G),5
9,594,"CONGELADO BATATA MCCAIN 9MM TRADICIONAL 2,5",5


In [13]:
1106,469

(1106, 469)

(1106, 469)